# Import libraries

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# CSV Import from kaggle database: 

#### Data from MyAnimeList (MyAL): https://www.kaggle.com/marlesson/myanimelist-dataset-animes-profiles-reviews


#### Data from Crunchyroll (CR): https://www.kaggle.com/filipefilardi/crunchyroll-anime-ratings

In [35]:
csv_file_crunchy = "datasets/crunchyroll.csv"
csv_file_myal = "datasets/myanimelist.csv"

In [3]:
crunchy_df = pd.read_csv(csv_file_crunchy )
myal_df = pd.read_csv(csv_file_myal)

# Cleaning databases

## General cleaning: (1) Droping duplicates in column title, (2) set all titles in lower de cases, (3) remove non alphabeticals characters inside titles, and (4) normalize rate, and score information.

### 1 Dropping duplicates

In [4]:
crunchy_df.drop_duplicates(subset = "anime", keep = "first", inplace = True) 
myal_df.drop_duplicates(subset = "title", keep = "first", inplace = True) 

### 2 Set titles in lower case

In [5]:
crunchy_df["anime"] = crunchy_df['anime'].str.lower()
myal_df["title"] = myal_df['title'].str.lower()

### 3 Remove non-alphabeticals characters in titles

In [6]:
crunchy_df["anime"] = crunchy_df['anime'].astype(str).str.replace('[^a-z,0-9, " "]', '')
myal_df["title"] = myal_df['title'].astype(str).str.replace('[^a-z,0-9," "]', '')

### 4 Normalize rate and score information to match

In [7]:
crunchy_df['rate'] = crunchy_df['rate']/5 *100
myal_df['score'] = myal_df['score']/10 *100

## Especific cleaning per dataframe: (1) Cleaning CR, and (2) Cleaning MyAL

### 1 Cleaning CR - transpose genre information to be able to compare it with the MyAnimeList data

#### Inspecting columns of the data

In [8]:
crunchy_df.columns.values

array(['anime', 'anime_url', 'anime_img', 'episodes', 'votes', 'weight',
       'rate', 'rate_1', 'rate_2', 'rate_3', 'rate_4', 'rate_5',
       'genre_action', 'genre_adventure', 'genre_comedy', 'genre_drama',
       'genre_family', 'genre_fantasy', 'genre_food', 'genre_harem',
       'genre_historical', 'genre_horror', 'genre_idols', 'genre_isekai',
       'genre_jdrama', 'genre_magical girls', 'genre_martial arts',
       'genre_mecha', 'genre_music', 'genre_mystery',
       'genre_post-apocalyptic', 'genre_romance', 'genre_sci-fi',
       'genre_seinen', 'genre_sgdrama', 'genre_shojo', 'genre_shonen',
       'genre_slice of life', 'genre_sports', 'genre_supernatural',
       'genre_thriller'], dtype=object)

#### Transpose columns by genre to rows using melt function. 

In [9]:
crunchy = pd.melt(crunchy_df, id_vars = ['anime', 'anime_url', 'episodes', 'rate'], 
       value_vars = ['genre_action', 
                    'genre_adventure', 
                    'genre_comedy', 
                    'genre_drama',
                    'genre_family', 
                    'genre_fantasy', 
                    'genre_food', 'genre_harem', 
                    'genre_historical', 
                    'genre_horror', 
                    'genre_idols', 
                    'genre_isekai', 
                    'genre_jdrama', 
                    'genre_magical girls', 
                    'genre_martial arts', 
                    'genre_mecha', 
                    'genre_music', 
                    'genre_mystery', 
                    'genre_post-apocalyptic', 
                    'genre_romance', 
                    'genre_sci-fi', 
                    'genre_seinen', 
                    'genre_sgdrama', 
                    'genre_shojo', 
                    'genre_shonen', 
                    'genre_slice of life', 
                    'genre_sports', 
                    'genre_supernatural', 
                    'genre_thriller'
                   ])
crunchy.head()

,anime,anime_url,episodes,rate,variable,value
0,naruto shippuuden,https://www.crunchyroll.com/naruto-shippuden,500,96.2,genre_action,0.0
1,shugo chara,https://www.crunchyroll.com/shugo-chara,51,97.2,genre_action,0.0
2,bleach,https://www.crunchyroll.com/bleach,366,96.6,genre_action,0.0
3,naruto,https://www.crunchyroll.com/naruto,220,96.0,genre_action,0.0
4,skip beat,https://www.crunchyroll.com/skip-beat,0,98.2,genre_action,0.0


#### Delete value column and rename values inside the genre column without the 'genre_'. Rename columns to Title, url, score and genre. 

In [10]:
crunchy = crunchy.drop(['value'], axis = 1)
crunchy['variable'] = crunchy['variable'].str.replace('genre_', '').astype(str)
crunchy = crunchy.rename(columns = {"anime": "title", "anime_url": "url", "rate": "score", "variable": "genre"})

### 2 Cleaning MyAnimeList - first normal form for genre information

#### Inspecting columns of the data

In [11]:
myal_df.columns.values

array(['uid', 'title', 'synopsis', 'genre', 'aired', 'episodes',
       'members', 'popularity', 'ranked', 'score', 'img_url', 'link'],
      dtype=object)

#### Create lists to contain the information per anime per genre listed to start the normalization of genre

In [12]:
title = []
episodes = []
score = []
link = []
genre = []

#### Iterate each row to delete symbols inside genre values and divide them value. Also iterate each of these values to add independent title, episodes, score, and link.

In [13]:
for index, data in myal_df.iterrows():
    data_genre = data['genre'] 
    data_genre = data_genre.replace('[', '') 
    data_genre = data_genre.replace(']', '')
    data_genre = data_genre.split(',')
    
    for dg in data_genre:
        genre.append(dg)
        title.append(data['title'])
        episodes.append(data['episodes'])
        score.append(data['score'])
        link.append(data['link'])

#### Create a new dictionary to contain values recolected in the list that we created to create a new data frame.

In [14]:
dict = {'title': title, 'link': link, 'episodes': episodes, 'score': score, 'genre': genre} 
myal = pd.DataFrame(dict)

#### Exploration of new dataframe for MyAL.

In [15]:
myal.head()

,title,link,episodes,score,genre
0,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25.0,88.2,'Comedy'
1,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25.0,88.2,'Sports'
2,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25.0,88.2,'Drama'
3,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25.0,88.2,'School'
4,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25.0,88.2,'Shounen'


#### Delete symbols inside values of genre column and make them lower case

In [16]:
myal['genre'] = myal['genre'].str.replace("'", '').astype(str)
myal["genre"] = myal['genre'].str.lower()

# Transformation of data to match both dataframes

#### Add id_source to each dataframe

In [17]:
crunchy['source'] = 1
myal['source'] = 2

#### Re-inspect dataframes to review the information stored.

In [18]:
crunchy.head()

,title,url,episodes,score,genre,source
0,naruto shippuuden,https://www.crunchyroll.com/naruto-shippuden,500,96.2,action,1
1,shugo chara,https://www.crunchyroll.com/shugo-chara,51,97.2,action,1
2,bleach,https://www.crunchyroll.com/bleach,366,96.6,action,1
3,naruto,https://www.crunchyroll.com/naruto,220,96.0,action,1
4,skip beat,https://www.crunchyroll.com/skip-beat,0,98.2,action,1


In [20]:
myal.head()

,title,link,episodes,score,genre,source
0,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25.0,88.2,comedy,2
1,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25.0,88.2,sports,2
2,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25.0,88.2,drama,2
3,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25.0,88.2,school,2
4,haikyuu second season,https://myanimelist.net/anime/28891/Haikyuu_Se...,25.0,88.2,shounen,2


#### Rename link to url to have the same names for both dataframes

In [21]:
myal = myal.rename(columns={"link":"url"})

#### Inspect type of data of both dataframes

In [22]:
crunchy.dtypes

title        object
url          object
episodes      int64
score       float64
genre        object
source        int64
dtype: object

In [24]:
myal.dtypes

title        object
url          object
episodes    float64
score       float64
genre        object
source        int64
dtype: object

#### Match type of data by changing type of episodes in MyAL

In [25]:
myal = myal[myal['episodes'].notnull()]
myal['episodes'] = myal['episodes'].astype(int)

### Append both data frames, and reset index

In [26]:
merge_df =crunchy.append(myal, ignore_index = True)
merge_df.shape

(81698, 6)

#### Verify merge and inspect

In [27]:
merge_df.head()

,title,url,episodes,score,genre,source
0,naruto shippuuden,https://www.crunchyroll.com/naruto-shippuden,500,96.2,action,1
1,shugo chara,https://www.crunchyroll.com/shugo-chara,51,97.2,action,1
2,bleach,https://www.crunchyroll.com/bleach,366,96.6,action,1
3,naruto,https://www.crunchyroll.com/naruto,220,96.0,action,1
4,skip beat,https://www.crunchyroll.com/skip-beat,0,98.2,action,1


In [28]:
print(merge_df.columns)
print(merge_df.dtypes)

Index(['title', 'url', 'episodes', 'score', 'genre', 'source'], dtype='object')
title        object
url          object
episodes      int64
score       float64
genre        object
source        int64
dtype: object


# Create catalogs - base on the data

![Data Model from QuickDBD](image.png)

In [23]:
anime = merge_df[['title','episodes']]
anime.drop_duplicates('title', inplace = True)
anime.reset_index(drop=True, inplace = True)
anime

,title,episodes
0,naruto shippuuden,500
1,shugo chara,51
2,bleach,366
3,naruto,220
4,skip beat,0
...,...,...
16521,netjuu no susume special,1
16522,naruto x ut,1
16523,miira no kaikata,12
16524,kingsglaive final fantasy xv,1


In [24]:
genre = merge_df[['genre']]
genre = genre.drop_duplicates('genre')
genre = genre.reset_index(drop=True).rename(columns={'genre':'genre_name'})
genre

,genre_name
0,action
1,adventure
2,comedy
3,drama
4,family
...,...
89,shounen ai
90,police
91,shoujo
92,samurai


## Create link to PgAdmin

### NOTE: Created database as 'anime' and created tables as describe in schema

In [25]:
from password import key

In [26]:
conn = f"postgres:{key}@localhost:5432/anime_db"
engine = create_engine(f'postgresql://{conn}')

In [27]:
# Confirm tables
engine.table_names()

['anime', 'rating', 'source_site', 'genre_anime', 'genre']

## Load info to tables

In [28]:
anime.head()

,title,episodes
0,naruto shippuuden,500
1,shugo chara,51
2,bleach,366
3,naruto,220
4,skip beat,0


In [29]:
anime.to_sql(name='anime', con=engine, if_exists='append', index=False)
genre.to_sql(name='genre', con=engine, if_exists='append', index=False)

### Manually added information from source_site table


insert into source_site (id, source_name)
values (1, 'Crunchyroll'),
		(2, 'MyAnimeList');
		
select * from source_site;

In [41]:
merge_df_ag = merge_df[['title', 'genre']]
merge_df_ag.drop_duplicates(inplace=True)
merge_df_ag

,title,genre
0,naruto shippuuden,action
1,shugo chara,action
2,bleach,action
3,naruto,action
4,skip beat,action
...,...,...
81693,chuunibyou demo koi ga shitai kirameki no slap...,comedy
81694,chuunibyou demo koi ga shitai kirameki no slap...,drama
81695,chuunibyou demo koi ga shitai kirameki no slap...,romance
81696,chuunibyou demo koi ga shitai kirameki no slap...,school


In [32]:
table_anime = pd.read_sql_table('anime', engine)
table_anime

,id,title,episodes
0,1,naruto shippuuden,500
1,2,shugo chara,51
2,3,bleach,366
3,4,naruto,220
4,5,skip beat,0
...,...,...,...
33047,33048,netjuu no susume special,1
33048,33049,naruto x ut,1
33049,33050,miira no kaikata,12
33050,33051,kingsglaive final fantasy xv,1


In [33]:
table_genre = pd.read_sql_table('genre', engine)
table_genre

,id,genre_name
0,1,action
1,2,adventure
2,3,comedy
3,4,drama
4,5,family
...,...,...
183,184,shounen ai
184,185,police
185,186,shoujo
186,187,samurai


In [42]:
merge_df_ag = merge_df_ag.merge(table_anime, on='title')
merge_df_ag.rename(columns={'id':'id_anime'}, inplace=True)
merge_df_ag = merge_df_ag.merge(table_genre, left_on='genre',right_on='genre_name')
merge_df_ag.rename(columns={'id':'id_genre'}, inplace=True)
merge_df_ag

,title,genre,id_anime,episodes,id_genre,genre_name
0,naruto shippuuden,action,1,500,1,action
1,naruto shippuuden,action,1,500,95,action
2,naruto shippuuden,action,16527,500,1,action
3,naruto shippuuden,action,16527,500,95,action
4,shugo chara,action,2,51,1,action
...,...,...,...,...,...,...
324739,taka no tsume x megane super,samurai,29325,2,187,samurai
324740,yami no teio kyuuketsuki dracula,vampire,7860,1,94,vampire
324741,yami no teio kyuuketsuki dracula,vampire,7860,1,188,vampire
324742,yami no teio kyuuketsuki dracula,vampire,24386,1,94,vampire


In [44]:
genre_anime = merge_df_ag[['id_anime','id_genre']]
genre_anime.drop_duplicates()

,id_anime,id_genre
0,1,1
1,1,95
2,16527,1
3,16527,95
4,2,1
...,...,...
324739,29325,187
324740,7860,94
324741,7860,188
324742,24386,94


In [45]:
genre_anime.to_sql(name='genre_anime', con=engine, if_exists='append', index=False)

In [60]:
rating = merge_df.merge(table_anime, on='title')
rating = rating.rename(columns={'id':'id_anime'})
rating = rating[['score','source','id_anime']]
rating = rating.rename(columns={'score':'rating','source':'id_source'})
rating = rating.drop_duplicates(['id_anime','id_source'])
rating

,rating,id_source,id_anime
0,96.2,1,1
1,96.2,1,16527
58,82.0,2,1
59,82.0,2,16527
70,97.2,1,2
...,...,...,...
163379,75.0,2,33050
163384,75.6,2,16525
163385,75.6,2,33051
163386,75.6,2,16526


In [61]:
rating.to_sql(name='rating', con=engine, if_exists='append', index=False)